## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2023-01-11 15:10:55--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2023-01-11 15:10:55--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce2dbdca8afc41209b8c5a3da35.dl.dropboxusercontent.com/cd/0/get/B0XssNqwgYMpfO5JQtKDAXhlPOZ5EIFqnpZrW908kPraNjQQQyF5PsC9Kp_YzOgW7FQOYalJWlX9Nl2y9JWVIhJL3b2ofctTE7p_VEURVmnmD6JICEZF6tvG05myhpohDjv8nleCAuaIpbNCqkDKZlFvwoGDkCMxBUnuE9n5miZOtQ/file?dl=1# [following]
--2023-01-11 15:10:56--  https://uce2dbdca8afc41209b8c5a3da35.dl.dropboxusercontent.com/cd/0/get/B0XssNqwgYMpfO5JQtKDAXhlPOZ5EIFqnpZrW908kPraNjQQQyF5PsC9Kp_YzOgW7FQOYalJWlX9Nl2y9JWVIhJL3b2ofctTE7p_VEURVmnmD6JIC

In [140]:
import numpy as np
import os
datapath = "/Users/nikita/Documents/Coding/YandexNLP/nlp_course/Data"

data = list(open(datapath+"/quora.txt", encoding="utf-8"))
data[50].lower()

"what tv shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [7]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [9]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(line.lower()) for line in data]

# YOUR CODE

In [11]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [19]:
print([' '.join(row) for row in data_tok[:2]]) # joined tokens back into space-separated sentence

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [25]:
from gensim.models import Word2Vec, FastText
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [26]:
fasttext = FastText(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv

In [35]:
# now you can get word vectors !
fasttext.get_vector('anything')

array([ 3.0292847 ,  0.6064185 , -1.244117  , -3.808426  ,  4.507706  ,
        2.784939  , -2.8034887 ,  0.3088087 , -4.7388544 ,  2.9159803 ,
       -3.3739557 , -0.4806246 ,  1.259116  , -1.1239697 ,  0.97565454,
       -1.3916705 ,  1.9329599 ,  2.0662723 ,  0.07992268, -2.0993164 ,
       -2.289464  ,  0.8508729 , -1.8411219 , -1.6860374 ,  0.77587116,
       -1.0354708 , -2.0205202 ,  0.1551952 ,  3.1427646 ,  2.5732946 ,
       -0.21292947, -0.8101665 ], dtype=float32)

In [34]:
# or query similar words directly. Go play with it!
model.most_similar('name')

[('surname', 0.8073523044586182),
 ('signature', 0.7174354195594788),
 ('picture', 0.7137465476989746),
 ('title', 0.7130457758903503),
 ('friend', 0.684221088886261),
 ('response', 0.6809653639793396),
 ('address', 0.6722516417503357),
 ('summary', 0.6680483222007751),
 ('story', 0.6643567085266113),
 ('names', 0.6626631021499634)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [36]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [109]:
model.get_vector('hi')

array([ 6.9106e-01,  1.4597e-01, -3.3517e-01,  4.8231e-02, -6.1608e-01,
       -1.4180e-01,  4.1552e-01, -2.6095e-02, -5.2588e-01, -1.3978e-01,
        1.4192e-01,  6.4903e-02, -2.9111e+00, -1.9016e-01, -3.6696e-01,
       -1.8864e-01,  3.0571e-01, -5.5503e-01, -2.6367e-01, -5.2027e-01,
        5.7470e-01, -8.1808e-01, -9.7218e-01,  1.7467e-02,  2.5340e-03,
       -1.0814e+00,  6.2471e-01, -2.6300e-02,  4.2472e-01, -9.0976e-01,
        5.4082e-01,  7.0558e-02, -3.9258e-01,  2.0687e-01,  3.7001e-01,
       -2.4747e-01, -1.6059e-01,  7.1630e-01,  5.2610e-01,  1.6087e-01,
       -1.5809e+00,  7.7889e-01, -1.3631e-01,  5.7475e-01, -1.8892e-01,
        2.9119e-01,  3.0367e-01, -1.1275e+00, -3.4706e-02, -2.0520e-01,
        3.2973e-01, -2.0884e-02,  1.8084e-01, -5.2420e-01,  1.0588e+00,
        3.8381e-01,  2.1159e-01,  3.8984e-01,  1.2267e-01, -8.0787e-01,
        4.5536e-01,  1.4119e-01,  8.4443e-03,  4.7866e-02, -3.4413e-01,
        1.3397e-01, -8.0766e-01, -1.7335e-02, -2.5935e-01,  4.85

In [37]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [41]:
words = model.index_to_key[:1000] 

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [42]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])# YOUR CODE

In [43]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [91]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance

pca = PCA(n_components=2)
word_vectors_pca = pca.fit_transform(word_vectors)

word_vectors_pca -= word_vectors_pca.mean(0)
word_vectors_pca /= word_vectors_pca.std(0) # i.e. the mean and std of the x values and y values need to be zero and one, independently, respectively

In [92]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [93]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [106]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p2600', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [95]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = TSNE(n_components=2).fit_transform(word_vectors)
    #YOUR CODE

In [103]:
word_tsne -= word_tsne.mean(0)
word_tsne /= word_tsne.std(0)

In [104]:
word_tsne

array([[ 0.3508196 ,  0.8361318 ],
       [ 0.4546968 ,  0.8013383 ],
       [ 0.37136963,  0.82100654],
       ...,
       [ 1.5743967 , -1.4433126 ],
       [-0.62374544,  1.2573215 ],
       [-0.7443116 , -1.137104  ]], dtype=float32)

In [107]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p2862', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [116]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')

    toks = tokenizer.tokenize(phrase.lower())

    for tok in toks:
        try:
            vector += model.get_vector(tok)
        except KeyError:
            pass

    vector /= len(toks)
    
    return vector
        
    

In [119]:
vector[::10]

array([ 0.2936065 , -0.02361388,  0.08615012, -0.09250911, -0.94880205,
       -0.15343277,  0.04692376,  0.16605972,  1.264787  ,  0.07990123],
      dtype=float32)

In [118]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

AssertionError: 

In [121]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])
# YOUR CODE

In [122]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [123]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [124]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p3138', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [125]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [126]:
data_vectors.shape

(537272, 100)

In [137]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    query_vec = get_phrase_embedding(query)
    cosine_similarity = data_vectors@query_vec.transpose()
    np.argsort(cosine_similarity)
    


    
    return np.sort(cosine_similarity)

In [138]:
find_nearest(query="How do i enter the matrix?", k=10)

array([-1.7584147,  0.       ,  0.       , ..., 30.28084  , 30.284447 ,
       31.281826 ], dtype=float32)

In [ ]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.